In [1]:
"""Take output events from .aedat files and creates datasets for testing
"""

import os
os.chdir("/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic")

import numpy as np
from matplotlib import pyplot as plt
import h5py

from dynapseReservoir.networks.dynapseParameters import dynapseStructure
from dynapseReservoir.networks.reservoirsParameters import generalNetP, r64_1_1_P
from dynapseReservoir.networks.outputParameters import generalOutP
import DYNAPSETools.dynapseOutDecoder as DOD


plt.close("all")


folderName = "/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/caer/myoutputs/spikeForTrain/"
collectedSpikesAddress = '/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/reservoir_Demo/reservoirTraining/collectedSpikes/'


In [ ]:
# ====================== Import .AEDAT file

%matplotlib tk

allTs = []
allCoreID = []
allNeuronID = []

for core in [1, 2, 3]:
# for core in [1]:

    for part in [1,2,3,4]:
#     for part in [1]:
        
        print('core', core, 'part', part, 'neuron from ',(part-1)*64, 'to', part*64-1 )

        neuronInThisPart = np.arange((part-1)*64, (part)*64)
        
        aedatName = 'core' + str(core) +'_' + 'part' + str(part)
        
        fileName = folderName + aedatName + '.aedat'
        decEvents = DOD.import_events(fileName)
        decEvents = decEvents.normalize()
        # decEvents.plot_events()

        # ====================== Define Filter events Core 0
        filterCore0 = []
        # size = generalNetP["sizeTargetEncoding"] # WHAT IS sizeTargetEncoding??
        # U0, C0, N = generalNetP["targetEncodingPos"]
        # targetNeurons = [x for x in range(N, size + N)]
        # filterCore0 += targetNeurons

        # Filter events taking only events that are important for elaboration
        U0, C0, N1 = generalNetP["startTriggerPos"]
        U0, C0, N2 = generalNetP["stopTriggerPos"]
        # U0, C0, N3 = generalNetP["endStimulusPos"]
        # filterCore0 += [N1, N2, N3]
        filterCore0 += [N1, N2]

        # ====================== Define Filter events Core 1
        #%%
        size = r64_1_1_P["N_Res"]
        U0, C1, N = generalNetP["reservoirStartPos"]
        filterCore1 = [x for x in range(N, size + N)]
        filterCore2 = [x for x in range(N, size + N)]
        filterCore3 = [x for x in range(N, size + N)]

        #%%

        # ====================== Apply filter
        neuron_id_filter = [filterCore0, filterCore1,filterCore2,filterCore3]


        C2 = 2
        C3 = 3

        decFilteredEvents = decEvents.filter_events(chip_id = U0, core_id = [C0,C1,C2,C3], neuron_id = neuron_id_filter) # Take only events i need
        # decFilteredEvents.plot_events()





        # ====================== Extract experiments
        extractedEvents = decFilteredEvents.isolate_events_sets(startTriggerNeuron = generalNetP["startTriggerPos"],
                                                                stopTriggerNeuron = generalNetP["stopTriggerPos"],
                                                                maxNumber = None)

        experiment = extractedEvents[0]

        experiment.plot_events()

        'core' + str(core) +'_' + 'part' + str(part)
        
        relevantNeuronIndices = list(np.where(np.logical_and(experiment.neuron_id.astype(int) >= np.min(neuronInThisPart), experiment.neuron_id.astype(int) <= np.max(neuronInThisPart)))[0])
        
        relevantNeuronCoreIndices = list(np.where(experiment.core_id.astype(int) == core)[0])
        
        relevantIndices = list(set(relevantNeuronCoreIndices).intersection(relevantNeuronIndices))
        relevantIndices.sort()
                               
                               
        relevantCoreID = np.array([experiment.core_id[index] for index in relevantIndices  ])
        relevantNeuronID =  np.array([experiment.neuron_id[index] for index in relevantIndices  ])
        relevantTS =  np.array([experiment.ts[index] for index in relevantIndices  ])
        
        relevantTS
        
        allCoreID.extend(relevantCoreID)                       
        allNeuronID.extend(relevantNeuronID)                       
        allTs.extend(relevantTS - relevantTS[0])                       


core 1 part 1 neuron from  0 to 63
Read all data

Extracted 1 experiments
core 1 part 2 neuron from  64 to 127
Read all data

Extracted 1 experiments
core 1 part 3 neuron from  128 to 191
Read all data

Extracted 1 experiments
core 1 part 4 neuron from  192 to 255
Read all data

Extracted 1 experiments
core 2 part 1 neuron from  0 to 63
Read all data

Extracted 1 experiments


In [4]:
np.savetxt(collectedSpikesAddress + 'collected_core_id.txt', allCoreID, fmt='%i')
np.savetxt(collectedSpikesAddress + 'collected_neuron_id.txt', allNeuronID, fmt='%i')
np.savetxt(collectedSpikesAddress + 'collected_ts.txt', allTs, fmt='%i')    


In [73]:
np.min(neuronInThisPart)

192

In [3]:
experiment.ts

array([ 4487461,  4487643,  4488135, ..., 35110320, 35110351, 35110521])

In [4]:
35110521/1e6

35.110521